<h3>Concrete model</h3>

This section shows how to create a concrete model for the following linear program:

$\max{z} = 30x_1 + 20x_2 + 50x_3$
<br>
<br>
Subject to:
<br>
<br>
$2x_1 + 3x_2 + 5x_3 \leq 4000$
<br>
$4x_1 + 2x_2 + 7x_3 \leq 6000$
<br>
$x_1 \geq 200$
<br>
$x_2 \geq 200$
<br>
$x_3 \geq 150$
<br>
$x_1, x_2, x_3 \in \Z_{+}$
<br>
<br>
The chosen solver is CPLEX by IBM.

In [8]:
from pyomo.core.base.var import Var
import pyomo.environ as pyo

model = pyo.ConcreteModel()
model.x = pyo.Var([1,2,3], domain=pyo.NonNegativeIntegers)
model.objectiveFunction = pyo.Objective(expr = 30*model.x[1] + 20*model.x[2] + 50*model.x[3], sense = pyo.maximize)

model.constraint1 = pyo.Constraint(expr = 2*model.x[1] + 3*model.x[2] + 5*model.x[3] <= 4000)
model.constraint2 = pyo.Constraint(expr = 4*model.x[1] + 2*model.x[2] + 7*model.x[3] <= 6000)
model.constraint3 = pyo.Constraint(expr = model.x[1] >= 200)
model.constraint4 = pyo.Constraint(expr = model.x[2] >= 200)
model.constraint5 = pyo.Constraint(expr = model.x[3] >= 150)

# outputs a summary of the model
model.pprint()

1 Set Declarations
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :  None :  None : False :  True : NonNegativeIntegers
          2 :     0 :  None :  None : False :  True : NonNegativeIntegers
          3 :     0 :  None :  None : False :  True : NonNegativeIntegers

1 Objective Declarations
    objectiveFunction : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 30*x[1] + 20*x[2] + 50*x[3]

5 Constraint Declarations
    constraint1 : Size=1, Index=None, Active=True
        Key  : Lower : Body                     : Upper  : Active
        None :  -Inf : 2*x[1] + 3*x[2] + 5*x[3] : 4000.0 :   True
    constraint2 : Size=1, Index=None, Active=True
        Key  : Lower : Body                     : 

Now we can pass our model to a solver (CPLEX in this case) to get a result:

In [6]:
solver = pyo.SolverFactory('cplex')
result = solver.solve(model)

# show results for the solved model
model.display()

Model unknown

  Variables:
    x : Size=3, Index=x_index
        Key : Lower : Value  : Upper : Fixed : Stale : Domain
          1 :     0 : 1044.0 :  None : False : False : NonNegativeIntegers
          2 :     0 :  387.0 :  None : False : False : NonNegativeIntegers
          3 :     0 :  150.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    objectiveFunction : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 46560.0

  Constraints:
    constraint1 : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 3999.0 : 4000.0
    constraint2 : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 6000.0 : 6000.0
    constraint3 : Size=1
        Key  : Lower : Body   : Upper
        None : 200.0 : 1044.0 :  None
    constraint4 : Size=1
        Key  : Lower : Body  : Upper
        None : 200.0 : 387.0 :  None
    constraint5 : Size=1
        Key  : Lower : Body  : Upper
        None : 150.0 : 150.0 :  None


In [7]:
model.solutions.load_from(result)

for v in model.component_objects(Var, active=True):
    print ("Variable",v)
    varobject = getattr(model, str(v))
    for index in varobject:
        print ("\t",index, varobject[index].value)


Variable x
	 1 1044.0
	 2 387.0
	 3 150.0
